In [1]:
import pandas as pd
import glob
from cleaning_functions import *

In [2]:
# Paths
# check if this is the same path used for raw_analysis
path = '../data/originals/*'

In [3]:
# Get a list of files to read from the path
files = glob.glob(path)

- Get the file path, the supplier_code (a key in the dictionary below)
and the parameters of the dictionary below for each key. It checks the file extension and then uses the appropriate pandas function.

In [4]:
list1 = []
def read_file(file, supplier_code, header, special_operation):
        # Check if file is xls or xlsx
    if "xlsx" in file or "xls" in file.lower():
        df = pd.read_excel(
            io = Fr"{file}", 
            header = file_columns[supplier_code]["header"], 
            usecols = file_columns[supplier_code]["columns"]
            )[file_columns[supplier_code]["columns"]] # To maintain parsed columns orders if the dataframe require more special operations like adding blank columns or removing unwanted rows
        list1.append(file_columns[supplier_code])
        if file_columns[supplier_code]["special_operation"]:
            df = file_columns[supplier_code]["special_operation"](df)

        df.columns = ["Manufacturer", "Partnumber", "Quantity", "Price"]

        df["supplier"] = supplier_code

        #Check if file is csv or txt
    elif "csv" in file.lower() or "txt" in file.lower():
        df = pd.read_csv(
            file, 
            header = file_columns[supplier_code]["header"], 
            usecols = file_columns[supplier_code]["columns"],
            sep = ";|\|", 
            encoding = "ISO-8859-1", # This encoding is important to work with most of the files
            engine = "python" # This engine is to accept regex at sep parameter for csv with variety of delimiters
            )[file_columns[supplier_code]["columns"]] # To maintain parsed columns orders

        if file_columns[supplier_code]["special_operation"]:
            df = file_columns[supplier_code]["special_operation"](df)

        df.columns = ["Manufacturer", "Partnumber", "Quantity", "Price"]

        df["supplier"] = supplier_code

    else:
        print("unkown file formate: ", file)

    return df

Dictionary to pass to the read function. For each file, it needs the header (where the column name is), the columns
names and if any special operations is needed (the cleaning_functions functions)

In [11]:

file_columns = {
    "metal":    {    "header": 0, 
                    "columns": ["Fabricante", "Codigo_fornecedor", "quantity", "PRECO_LIQUIDO"],
                    "special_operation": None
                },

    "medauto":    {    "header": 0, 
                    "columns": ["Fornecedor", "NUMERO DA PEÇA", "ESTOQUE", "PREÇO"],
                    "special_operation": None
                },
    "lucios":     {    "header": 0, 
                    "columns": ["Fabricante", "Código Lucios", "Disponibilidade", "Preço"],
                    "special_operation": None
                },
    "carbwel":     {    "header": 0, 
                    "columns": ["Fabricante", "Código Fabricante", "Disponivel", "PrcVenda"],
                    "special_operation": None
                },
    "mte":     {    "header": 0, 
                    "columns": ["COD MTE", "QTD. ESTOQUE", "PREÇO"],
                    "special_operation": mte_process
                },
    "sueyasu":     {    "header": 0, 
                    "columns": ["NOME DO FABRICANTE", "CODIGO DA PEÇA (FABRICANTE)", "QUANTIDADE EM ESTOQUE", "PREÇO"],
                    "special_operation": sueyasu_process
                },
    "polipecas":{    "header": 2, 
                    "columns": ["COD. FORNECEDOR", "PREÇO"],
                    "special_operation": polipecas_process
                },
    "lucios":{        "header": 0, 
                    "columns": ["Fabricante", "Código Fábrica", "Disponibilidade", "Preço"],
                    "special_operation": None
                },
    "ima":{            "header": 3, 
                    "columns": ["Código","Preço C/Imp SP", "Múltiplos"],
                    "special_operation": ima_process
                },
    "compel":{        "header": 0, 
                    "columns": ["FABRICANTE", "COD FABRICA", "DISPONIBILIDADE", "PREÇO"],
                    "special_operation": None
                },
    "rufato":{        "header": None, 
                    "columns": [5, 4, 3, 2],
                    "special_operation": None
                },
    "real":{        "header": 0, 
                    "columns": ["CODIGO_INTERNO", "COD_FABRICANTE","NOME_FANTASIA","PRODUTO",
                                "QTDE_EMB","UNIDADE_MEDIDA","QTDE_SPNORTE","PRECO_SPNORTE",
                                "QTDE_ABC","PRECO_ABC","QTDE_SPLESTE","PRECO_SPLESTE"],
                    "special_operation": real_process
                },
    "jahu":{        "header": 0, 
                    "columns": ['Marca','Cód.Fabricante', 'Preco','Disponivel',"Produto"],
                    "special_operation": jahu_process
                },
}


Starts with an empty list that is populated with every data from the files list using the read_file function

In [12]:
all_df = []
for file in files:
    file_columns_keys = list(file_columns.keys())
    for supplier_code in file_columns_keys:
        if supplier_code in file.lower():
            print(file, "====", supplier_code, file_columns[supplier_code]["header"])
            all_df.append(read_file( #dataframes are added one by one and merged all at once
                file, 
                supplier_code, 
                header = file_columns[supplier_code]["header"], 
                special_operation = file_columns[supplier_code]["special_operation"]))

../data/originals\00-Prod_Metal_001.xlsx ==== metal 0
../data/originals\2022-03-01 - Lista de Preços IMA - 03-2022 - GERAL.XLSX ==== ima 3
../data/originals\ARQUIVO-RUFATO-29-11-2022-HR12-00.TXT ==== rufato None
../data/originals\Arquivo_Compel-13-01-23.csv ==== compel 0
../data/originals\Estoque Carbwel 20 10 22 Karhub.xls ==== carbwel 0
../data/originals\Estoque real 29 11.csv ==== real 0


C:\Users\eppmi\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
E:\Dropbox\Pessoal\Python\trabalho\Upwork\New-stock-system\data_merge\cleaning_functions.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].apply(lambda x: x.replace(',','.'))


../data/originals\estoque_jahu_20221129.csv ==== jahu 0
../data/originals\medauto_stock_220916.xlsx ==== medauto 0


E:\Dropbox\Pessoal\Python\trabalho\Upwork\New-stock-system\data_merge\cleaning_functions.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Cód.Fabricante"] = df2["Produto"]


../data/originals\MTE_ESTOQUE_INTERNET-2.XLSX ==== mte 0
../data/originals\polipecas_stock_220919.xlsx ==== polipecas 2
../data/originals\Relatorio-lucios.xls ==== lucios 0
WARNING *** file size (3141173) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
../data/originals\Sueyasu - 27-09-22.xlsx ==== sueyasu 0


In [13]:
pd.concat(all_df, ignore_index = True).to_csv("../data/merged_database/combined.csv")

In [14]:
# save the output file in third_etl in the raw_stock folder, naming it with real_third_etl
pd.concat(all_df, ignore_index = True).to_csv("../data/raw_analysis/real_third_etl.csv")

In [15]:
#Save with timestamp
pd.concat(all_df, ignore_index = True).to_csv('../data/text_output/real_third_etl_{}.txt'.format(pd.datetime.now().strftime("%Y-%m-%d %H-%M-%S")))

C:\Users\eppmi\AppData\Local\Temp/ipykernel_16848/1044509951.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.concat(all_df, ignore_index = True).to_csv('../data/text_output/real_third_etl_{}.txt'.format(pd.datetime.now().strftime("%Y-%m-%d %H-%M-%S")))
